In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np

In [6]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                    optimizer=optimizer,
                                    metrics=['accuracy'])

        # Build the generator
        self.generator = self.generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.show()
        fig.savefig("images/%d.png" % epoch)
        plt.close()



In [7]:
gan = GAN()
gan.train(epochs=2000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

C:\Users\ziaeeamir\AppData\Local\Continuum\anaconda3\envs\Huber\lib\site-packages\ipykernel_launcher.py:140: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
C:\Users\ziaeeamir\AppData\Local\Continuum\anaconda3\envs\Huber\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.493129, acc.: 71.88%] [G loss: 0.781995]
2 [D loss: 0.333483, acc.: 85.94%] [G loss: 0.878936]
3 [D loss: 0.288671, acc.: 90.62%] [G loss: 0.963858]
4 [D loss: 0.239660, acc.: 96.88%] [G loss: 1.126134]
5 [D loss: 0.221826, acc.: 93.75%] [G loss: 1.189194]
6 [D loss: 0.191228, acc.: 100.00%] [G loss: 1.369176]
7 [D loss: 0.169403, acc.: 100.00%] [G loss: 1.384684]
8 [D loss: 0.167435, acc.: 98.44%] [G loss: 1.478073]
9 [D loss: 0.149959, acc.: 98.44%] [G loss: 1.640492]
10 [D loss: 0.124742, acc.: 100.00%] [G loss: 1.741439]
11 [D loss: 0.116459, acc.: 100.00%] [G loss: 1.785216]
12 [D loss: 0.113995, acc.: 100.00%] [G loss: 1.923182]
13 [D loss: 0.087438, acc.: 100.00%] [G loss: 1.978010]
14 [D loss: 0.095532, acc.: 100.00%] [G loss: 2.045919]
15 [D loss: 0.073186, acc.: 100.00%] [G loss: 2.084229]
16 [D loss: 0.077917, acc.: 100.00%] [G loss: 2.152665]
17 [D loss: 0.074424, acc.: 100.00%] [G loss: 2.189111]
18 [D loss: 0.070047, acc.: 100.00%] [G loss: 2.354005]
19 [D lo

149 [D loss: 0.268147, acc.: 85.94%] [G loss: 3.527683]
150 [D loss: 0.141805, acc.: 93.75%] [G loss: 4.204290]
151 [D loss: 0.935736, acc.: 60.94%] [G loss: 1.991355]
152 [D loss: 0.444518, acc.: 79.69%] [G loss: 2.634040]
153 [D loss: 0.092960, acc.: 96.88%] [G loss: 4.118455]
154 [D loss: 0.169482, acc.: 95.31%] [G loss: 3.223197]
155 [D loss: 0.126266, acc.: 95.31%] [G loss: 3.474217]
156 [D loss: 0.078446, acc.: 100.00%] [G loss: 3.511288]
157 [D loss: 0.207887, acc.: 89.06%] [G loss: 4.109053]
158 [D loss: 0.398187, acc.: 81.25%] [G loss: 2.728858]
159 [D loss: 0.298288, acc.: 82.81%] [G loss: 4.037772]
160 [D loss: 0.055666, acc.: 98.44%] [G loss: 4.480571]
161 [D loss: 0.649428, acc.: 73.44%] [G loss: 2.436865]
162 [D loss: 0.193931, acc.: 87.50%] [G loss: 3.452744]
163 [D loss: 0.052863, acc.: 100.00%] [G loss: 4.192307]
164 [D loss: 0.333577, acc.: 81.25%] [G loss: 3.323624]
165 [D loss: 0.071074, acc.: 100.00%] [G loss: 3.883965]
166 [D loss: 0.137064, acc.: 100.00%] [G loss

298 [D loss: 0.719621, acc.: 42.19%] [G loss: 0.978294]
299 [D loss: 0.783890, acc.: 40.62%] [G loss: 0.898425]
300 [D loss: 0.807649, acc.: 42.19%] [G loss: 0.760340]
301 [D loss: 0.675511, acc.: 50.00%] [G loss: 1.012598]
302 [D loss: 0.907948, acc.: 29.69%] [G loss: 0.713634]
303 [D loss: 0.650772, acc.: 59.38%] [G loss: 0.810377]
304 [D loss: 0.732735, acc.: 45.31%] [G loss: 0.892612]
305 [D loss: 0.821824, acc.: 29.69%] [G loss: 0.817252]
306 [D loss: 0.691498, acc.: 48.44%] [G loss: 0.795027]
307 [D loss: 0.780911, acc.: 43.75%] [G loss: 0.740400]
308 [D loss: 0.758489, acc.: 40.62%] [G loss: 0.727806]
309 [D loss: 0.677973, acc.: 51.56%] [G loss: 0.858028]
310 [D loss: 0.730166, acc.: 45.31%] [G loss: 0.821460]
311 [D loss: 0.734929, acc.: 48.44%] [G loss: 0.730466]
312 [D loss: 0.734680, acc.: 45.31%] [G loss: 0.706012]
313 [D loss: 0.709679, acc.: 48.44%] [G loss: 0.782387]
314 [D loss: 0.698119, acc.: 48.44%] [G loss: 0.820860]
315 [D loss: 0.706973, acc.: 51.56%] [G loss: 0.

...

1328 [D loss: 0.591478, acc.: 65.62%] [G loss: 0.962249]
1329 [D loss: 0.561046, acc.: 82.81%] [G loss: 0.987107]
1330 [D loss: 0.556749, acc.: 78.12%] [G loss: 1.005787]
1331 [D loss: 0.583784, acc.: 65.62%] [G loss: 0.986243]
1332 [D loss: 0.527939, acc.: 78.12%] [G loss: 1.007987]
1333 [D loss: 0.564083, acc.: 75.00%] [G loss: 0.971187]
1334 [D loss: 0.554870, acc.: 78.12%] [G loss: 0.955423]
1335 [D loss: 0.526217, acc.: 82.81%] [G loss: 0.967679]
1336 [D loss: 0.552836, acc.: 75.00%] [G loss: 1.038431]
1337 [D loss: 0.531498, acc.: 84.38%] [G loss: 0.973570]
1338 [D loss: 0.537719, acc.: 76.56%] [G loss: 1.024739]
1339 [D loss: 0.587404, acc.: 73.44%] [G loss: 0.913974]
1340 [D loss: 0.595364, acc.: 65.62%] [G loss: 0.969297]
1341 [D loss: 0.553952, acc.: 75.00%] [G loss: 0.967828]
1342 [D loss: 0.580168, acc.: 67.19%] [G loss: 0.929983]
1343 [D loss: 0.561627, acc.: 71.88%] [G loss: 0.905027]
1344 [D loss: 0.559753, acc.: 75.00%] [G loss: 0.944581]
1345 [D loss: 0.566988, acc.: 7

1474 [D loss: 0.638890, acc.: 62.50%] [G loss: 0.951663]
1475 [D loss: 0.575442, acc.: 75.00%] [G loss: 0.912192]
1476 [D loss: 0.569004, acc.: 78.12%] [G loss: 0.924028]
1477 [D loss: 0.575687, acc.: 71.88%] [G loss: 0.884948]
1478 [D loss: 0.618215, acc.: 70.31%] [G loss: 0.868027]
1479 [D loss: 0.555196, acc.: 73.44%] [G loss: 0.990083]
1480 [D loss: 0.585455, acc.: 68.75%] [G loss: 0.923452]
1481 [D loss: 0.592425, acc.: 71.88%] [G loss: 0.920076]
1482 [D loss: 0.570298, acc.: 73.44%] [G loss: 0.912494]
1483 [D loss: 0.580140, acc.: 71.88%] [G loss: 1.001856]
1484 [D loss: 0.578627, acc.: 71.88%] [G loss: 0.937409]
1485 [D loss: 0.682560, acc.: 56.25%] [G loss: 0.942109]
1486 [D loss: 0.573210, acc.: 78.12%] [G loss: 0.948162]
1487 [D loss: 0.590796, acc.: 65.62%] [G loss: 0.922668]
1488 [D loss: 0.572448, acc.: 73.44%] [G loss: 0.988629]
1489 [D loss: 0.585836, acc.: 68.75%] [G loss: 0.967847]
1490 [D loss: 0.599055, acc.: 67.19%] [G loss: 0.912740]
1491 [D loss: 0.503643, acc.: 7

1617 [D loss: 0.591706, acc.: 71.88%] [G loss: 0.892395]
1618 [D loss: 0.501663, acc.: 90.62%] [G loss: 0.956453]
1619 [D loss: 0.567742, acc.: 70.31%] [G loss: 0.936830]
1620 [D loss: 0.591344, acc.: 70.31%] [G loss: 0.902258]
1621 [D loss: 0.582256, acc.: 71.88%] [G loss: 0.936302]
1622 [D loss: 0.573324, acc.: 75.00%] [G loss: 0.961616]
1623 [D loss: 0.529609, acc.: 78.12%] [G loss: 0.934957]
1624 [D loss: 0.534974, acc.: 73.44%] [G loss: 0.896875]
1625 [D loss: 0.548368, acc.: 68.75%] [G loss: 0.951632]
1626 [D loss: 0.668001, acc.: 57.81%] [G loss: 0.907756]
1627 [D loss: 0.572323, acc.: 78.12%] [G loss: 0.945330]
1628 [D loss: 0.711776, acc.: 51.56%] [G loss: 0.949167]
1629 [D loss: 0.581188, acc.: 71.88%] [G loss: 0.983682]
1630 [D loss: 0.619860, acc.: 60.94%] [G loss: 0.934282]
1631 [D loss: 0.589053, acc.: 64.06%] [G loss: 0.910657]
1632 [D loss: 0.603643, acc.: 71.88%] [G loss: 0.939171]
1633 [D loss: 0.609989, acc.: 67.19%] [G loss: 0.879547]
1634 [D loss: 0.559916, acc.: 7

1763 [D loss: 0.580767, acc.: 71.88%] [G loss: 0.973884]
1764 [D loss: 0.644823, acc.: 68.75%] [G loss: 0.894904]
1765 [D loss: 0.619586, acc.: 65.62%] [G loss: 0.910577]
1766 [D loss: 0.648547, acc.: 54.69%] [G loss: 0.883452]
1767 [D loss: 0.579568, acc.: 70.31%] [G loss: 0.933232]
1768 [D loss: 0.647698, acc.: 62.50%] [G loss: 0.892861]
1769 [D loss: 0.633253, acc.: 64.06%] [G loss: 0.967495]
1770 [D loss: 0.567529, acc.: 73.44%] [G loss: 0.927405]
1771 [D loss: 0.623709, acc.: 62.50%] [G loss: 0.831098]
1772 [D loss: 0.658179, acc.: 68.75%] [G loss: 0.842340]
1773 [D loss: 0.629161, acc.: 73.44%] [G loss: 0.961286]
1774 [D loss: 0.615036, acc.: 73.44%] [G loss: 0.894375]
1775 [D loss: 0.631727, acc.: 67.19%] [G loss: 0.960403]
1776 [D loss: 0.630492, acc.: 64.06%] [G loss: 0.901940]
1777 [D loss: 0.616301, acc.: 68.75%] [G loss: 0.883617]
1778 [D loss: 0.596072, acc.: 68.75%] [G loss: 0.894747]
1779 [D loss: 0.590723, acc.: 68.75%] [G loss: 0.950339]
1780 [D loss: 0.626307, acc.: 6

1908 [D loss: 0.571909, acc.: 73.44%] [G loss: 0.924031]
1909 [D loss: 0.608976, acc.: 67.19%] [G loss: 1.028852]
1910 [D loss: 0.582776, acc.: 71.88%] [G loss: 0.942974]
1911 [D loss: 0.632395, acc.: 70.31%] [G loss: 0.894107]
1912 [D loss: 0.596630, acc.: 68.75%] [G loss: 0.949951]
1913 [D loss: 0.610966, acc.: 68.75%] [G loss: 0.893254]
1914 [D loss: 0.615850, acc.: 65.62%] [G loss: 0.877757]
1915 [D loss: 0.571992, acc.: 71.88%] [G loss: 0.904830]
1916 [D loss: 0.625341, acc.: 65.62%] [G loss: 0.940656]
1917 [D loss: 0.575008, acc.: 68.75%] [G loss: 0.961418]
1918 [D loss: 0.596569, acc.: 68.75%] [G loss: 0.909103]
1919 [D loss: 0.541844, acc.: 78.12%] [G loss: 0.957188]
1920 [D loss: 0.573523, acc.: 75.00%] [G loss: 0.945732]
1921 [D loss: 0.645249, acc.: 65.62%] [G loss: 1.008186]
1922 [D loss: 0.579421, acc.: 65.62%] [G loss: 0.929888]
1923 [D loss: 0.548820, acc.: 79.69%] [G loss: 0.995279]
1924 [D loss: 0.588704, acc.: 67.19%] [G loss: 0.951089]
1925 [D loss: 0.587196, acc.: 6